# Entornos de programación de audio/música


- Muchos entornos para síntesis, efectos y composición. 

- Algunos incluyen lenguajes propios de programación o librerías para otros lenguajes.
    
    - Generadores de señal: osciladores, FM, ruido. . .

    - Procesadores de señal (DSP): envolventes, filtros, FFT, etc.

    - Pueden incluir entorno gráfico, acceso midi, exportación a VSTs (LV2, etc)...

    - Actualmente permiten síntesis en tiempo real, facilidades para composición algorítmica... investigación de sonido... y facilidades para live coding music

Una lista: https://en.wikipedia.org/wiki/Comparison_of_audio_synthesis_environments

---


Algunos **ecosistemas/entornos** de código abierto

- Csound basado en C (heredero directo de Music V, Max Mathews)

- Chuck, con las STK (librerías de **modelado físico**)

- Pure Data (versión libre de MAX/MSP): entorno gráfico orientado a NO programadores

- Extempore (hermano de Impromptu): basado en lisp, para live music coding

- Faust: potentísimo lenguaje minimal, muy completo para programación de síntesis y efectos: exporta a JUCE, Supercollider...

- **Supercollider**

- **Pyo**: "versión Python" de SuperCollider

- JUCE, librería de desarrollo


Otros comerciales

- MAX/MSP

- Reaktor (Native Instruments)

- Kyma (Korg)

# Pyo

Misma filosofía que SuperCollider

- Programado en C (muy eficiente)

- Con interfaz en Python (muy cómodo de usar)

Instalación: ```pip install pyo```

- Última version 1.0.5

    https://belangeo.github.io/pyo/gettingstarted.html

    (ojo, hay páginas web desactualizadas)

- Para acceder a las últimas versiones puede ser necesario compilar fuentes (al menos en Linux)


# Arrancando Pyo

```
from pyo import *

s = Server()
s.boot() 
s.start()
```

- Al crear el servidor y arrancarlo (boot):

    - Abre los interfaces de audio y de midi

    - Configura numero de canales (**nchnls=2**), sample rate (**sr=44100**), tamaño del buffer (**buffersize=128**, chunk size), etc

- Al iniciarlo (start), empieza a procesar audio.

In [ ]:
# inicializacion de pyo y primer sonido

from pyo import *

# s = Server() # opciones por defecto
s = Server(audio="jack") # driver jack (linux). Otros paráms: sr=44100, nchnls=2, buffersize=128, duplex=1

s.boot() # inicio del servidor

# oscilador de seno
a = Sine(freq=180, mul=0.1)

# se envia a la salida de audio
a.out()

s.start() # arranque del servidor


In [ ]:
# para parar el servidor (en modo interactivo con notebooks)

s.stop()


In [ ]:
# Abreviado y con tiempo programado de 3 segundos

from pyo import *

s = Server(audio="jack").boot().start()

a = Sine(freq=110, mul=0.1).out()

# dejamos sonar 3 segundos y terminamos
import time
time.sleep(3)
s.stop()

# GUI en Pyo

Pyo proporciona una GUI muy cómoda para interactuar con el sistema, que se invoca con ```s.gui(locals())```

- Lanza una ventana que toma el **control de la ejecución** (análogo a TkInter): no escribir código tras esta llamada!!

    - Permite arrancar y parar el servidor (star/stop) y grabar audio (en un archivo prefijado)

    - La ventana de texto de abajo es un intérprete para interactuar con el servidor

- ```locals()``` obtiene las variables *locales* del programa

    - Las que estén definidas para control, se visualizan gráficamente



In [ ]:
# Inciso, variables locales del sistema
from pyo import *
s = Server(audio="jack",sr=48000).boot()

osc = Sine(110).out()

# todas las vars locales del Python
for l in list(locals()):    
    print(l,end='\n')

# la variables osc
print(locals()['osc'])

In [ ]:
# GUIs. 
# el start() va en la ventana de control, pero tb puede ponerse aquí

from pyo import *
s = Server(audio="jack",sr=48000).boot()
s.amp = 0.2 # control de volumen (master)

# sin a 110 Hz, amplitud 0.1
osc = Sine(110,mul=0.1)

# enrutamos señal a la salida de audio
osc.out()

# tb se puede abreviar 
#osc = Sine(110,mul=0.1).out()

# abrimos ventana de control interactiva
osc.ctrl(title="Seno simple")

s.gui(locals())

In [ ]:
# Síntesis aditiva con control conjunto de frecuencias
from pyo import *
s = Server(audio="jack",sr=48000).boot()
s.amp = 0.1 

oscs = Sine([100, 200, 300, 400, 500, 600, 700, 800],mul=0.2).out()

oscs.ctrl(title="Osciladores")

s.gui(locals())


In [ ]:
# Síntesis aditiva con control independiente de senos

from pyo import *
s = Server(audio="jack",sr=48000).boot()
s.amp = 0.1 

N = 4
oscs = [Sine(freq = 100*(i+1),mul=0.2).out() for i in range(N)]
[oscs[i].ctrl(title=f"Sine {i}") for i in range(N)]


s.gui(locals())

In [ ]:
# Moduladores: controlando el rango de salida de la señal

from pyo import *; s = Server(audio="jack",sr=48000).boot()

# The `mul` attribute multiplies each sample by its value.
a = Sine(freq=100, mul=0.1)

# The `add` attribute adds an offset to each sample.
# The multiplication is applied before the addition.
b = Sine(freq=100, mul=0.5, add=0.5)

# Using the range(min, max) method allows to automatically
# compute both `mul` and `add` attributes.
c = Sine(freq=100).range(-0.25, 0.5)

# Displays the waveforms
sc = Scope([a, b, c])

s.gui(locals())

# Expansion multicanal

Generación de múltiples *streams* de audio

- Un *stream* es un canal **mono** de salida de audio. Si ```s``` es un stream:

    - ```s.out()``` dirige la salida a un solo canal (equivalente a ```.out(0)```)
    - ```s.out(0)```,  ```s.out(1)``` -> izquierdo, derecho

- Para salida por los dos canales:
    
    - ```b = a.mix(voices=2).out()``` funciona como un *panner*
        - abreviado ```b = a.mix(2).out()``` 

    - ```b = a.out(chnl=[0,1])``` envío explícito a los dos canales



In [ ]:
# Paralelización de señales

from pyo import *; s = Server(audio="jack",sr=48000).boot(); s.amp=0.1

# Lista de armónicos de 1 a 12, solo los impares!
harms = [110 * i for i in range(1, 12) if i % 2 == 1]

# Amplitudes (1 / n) para esos armónicos
amps = [0.33 / i for i in range(1, len(harms)) if i % 2 == 1]

# Todos los osciladores a la vez!
a = Sine(freq=harms, mul=amps)
print(len(a))

# Mezclamos todas las voces/streams y sacamos a dos canales
b = a.out(chnl=[0,1])  # equivalente b = a.mix(voices=2).out()

sc = Scope(b)
s.gui(locals())

In [ ]:
# Serialización de señales

from pyo import *
s = Server(audio="jack",sr=48000).boot()
s.amp=0.1

# la salida de señal sigue la ruta a -> h1 -> h2 -> h3 -> h4 -> out 
# y todas ellas a su vez van al out
a = Sine() 
h1 = Harmonizer(a) 
h2 = Harmonizer(h1)
h3 = Harmonizer(h2)
h4 = Harmonizer(h3).out(chnl=[0,1])

a.ctrl()

s.gui(locals())

In [ ]:
# ruido: interpolación entre tres tipos de ruido

from pyo import *
s = Server(audio="jack",sr=48000).boot()
s.amp=0.1


# White noise
n1 = Noise(0.3)

# Pink noise
n2 = PinkNoise(0.3)

# Brown noise
n3 = BrownNoise(0.3)

# Interpolates between input objects to produce a single output
sel = Selector([n1, n2, n3]).out()
sel.ctrl(title="Input interpolator (0=White, 1=Pink, 2=Brown)")

# Displays the spectrum contents of the chosen source
sp = Spectrum(sel)

s.gui(locals())

In [ ]:
# filtros
from pyo import * ; s = Server(audio="jack",sr=48000).boot(); s.amp=0.1

# White noise
n = Noise(0.3)

# filtro LP (tipo butterworth)
lp = ButLP(n).out(chnl=[0,1])  # dur=10, delay=5)
lp.ctrl()

sp = Spectrum(lp)
s.gui(locals())

In [ ]:
# filtro comb (idea para síntesis sustractiva)
from pyo import * ; s = Server(audio="jack",sr=48000).boot(); s.amp=0.1

# White noise
n = Noise(0.3)

#t = SquareTable(order=15).normalize()
#n = Osc(table=t, freq=[220], mul=.2)

fl = IRPulse(n, freq=220, bw=1100, type=0, order=256, mul=1, add=0).out(chnl=[0,1])
fl.ctrl()

sp = Spectrum(fl)
s.gui(locals())



In [ ]:
# cargando de archivos y reproducción

from pyo import *
s = Server(audio="jack",sr=48000).boot()
s.amp=0.1

path = SNDS_PATH + "/transparent.aif"

# stereo playback with a slight shift between the two channels.
sf = SfPlayer(path, speed=[1, 0.995], loop=True, mul=0.4).out()
# por qué reproduce en dos canales?? -> expansión multicanal! por el argumento de speed

sf.ctrl()

s.gui(locals())

In [ ]:
# floats y señales

from pyo import *; s = Server(audio="jack",sr=48000).boot(); s.amp=0.1

# numero (entrero o float)
anumber = 0.5

# no funciona, no es señal
# sc = scope([anumber])

# Conversion de numero a señal (vector de floats)
astream = Sig(anumber)

# señal constante
sc = Scope([astream])

# Use a Print (capital "P") object to print an audio stream.
pp = Print(astream, interval=1, message="Audio stream value")

# Use the get() method to extract a float from an audio stream.
print("Float from audio stream : ", astream.get())

s.gui(locals())

In [ ]:
# Chowning algoritmo con ratios libres

from pyo import * ; s = Server(audio="jack",sr=48000).boot(); s.amp=0.1

# FM implements the basic Chowning algorithm
fm = FM(carrier=250, ratio=[2,2], index=10, mul=0.2).out(chnl=[0,1])
fm.ctrl()

sc = Scope([fm])
s.gui(locals())

In [ ]:
# FM similar a la que impementamos con numpy
from pyo import *; s = Server(audio="jack",sr=48000).boot(); 
s.amp=0.1

# carrier, ratio e index, definidos como Sig (señal) para poder modificarlos dinámicamente
# y que se recalculen todas las funciones (los dos senos)
# Si se opera una señal por un numero se "pierde" como señal
car = Sig(110) 
ratio = Sig(1)
index = Sig(5)

# controles para estos parámetros, escalados a valores "naturales" en vez de [0,1]
#           SLMap(min, max, scale, name, init, res='float', ramp=0.025, dataOnly=False)
car.ctrl(  [SLMap(min=50,max=1200,scale='lin',name='value',init=110,res='float')],title='freq')
ratio.ctrl([SLMap(min=0,max=20,scale='lin',name='value',init=1,res='int',dataOnly=True)],title='ratio')
index.ctrl([SLMap(min=0,max=40,scale='lin',name='value',init=5,res='float')],title='index')

# modulador
mod = Sine(ratio*car)
# salida: ojo multiplicamos moduladore por index^2
signal = Sine(car+index*index*mod).out()

sc = Spectrum(signal)

s.gui(locals())


In [ ]:
from pyo import * ; s = Server(audio="jack",sr=48000).boot(); s.amp=0.1

# tabla de sedmentos lineales, para la envolvente
ind = LinTable([(0,20), (200,5), (300,2), (2000,0)])

# metrónomo
m = Metro(4).play()

# lanzamiento de envolvente acorde a metrónomo
tr = TrigEnv(m, table=ind, dur=4)


# FM implements the basic Chowning algorithm
fm = FM(carrier=250, ratio=[2,2], index=2, mul=tr).out(chnl=[0,1])
fm.ctrl()

sc = Scope([fm])
s.gui(locals())

In [ ]:
# delay

from pyo import * ; s = Server(audio="jack",sr=48000).boot(); s.amp=0.1

sample = SfPlayer(SNDS_PATH + "/transparent.aif", loop=True,mul=.3).mix(2).out()

delay = Delay(sample,delay=[.35,.4],feedback=.5,mul=.4).out()

s.gui(locals())


In [ ]:
# fm con adsr
from pyo import * ; s = Server(audio="jack",sr=48000).boot(); s.amp=0.2

# adsr como siempre. Play lanza el adsr
adsr = Adsr(attack=.05,decay=.5,sustain=.7,release=.4,dur=1.5).play()

# señal fm, modulaada por el adsr
fm = FM(carrier=220, ratio=[.2,.2], index=2, mul=adsr*0.6).mix(2).out()

s.gui(locals())



# Secuenciación



- Pattern()

- Metro()

- CallAfter() 

- Score()



- TrigEnv()

In [ ]:
# Utilización de patrones programados

from pyo import * ; s = Server(audio="jack",sr=48000).boot(); s.amp=0.2

# duradión a 0 -> sustain = 0
# no necesita stop
adsr = Adsr(attack=.05,decay=.1,sustain=.6,release=.1,dur=0)

fm = FM(carrier=200, ratio=.248, index=adsr*8,mul=adsr*0.3).mix(2).out()

def playNote(count=0):
    if count%2==0:
        adsr.play()
    else:
        adsr.stop()
    count +=1

# pasamo la función "play" como primer argumento
# llama a play cada 0.2 segundos
# .play() arranca el patron
pat = Pattern(playNote, time=0.2).play()

s.gui(locals())


In [ ]:
# Metro: metrónomo
# Señal que lanza pulsos: devuelve todo 0s, y un 1 cada unidad prefijada de tiempo

from pyo import * ; s = Server(audio="jack",sr=48000).boot(); s.amp=0.2

metro = Metro(time=1).play()

Scope([metro])

s.gui(locals())


In [ ]:
# Counter: contador de llamadas recibidas

from pyo import * ; s = Server(audio="jack",sr=48000).boot(); s.amp=0.2

# metronomo cada 1/2 secundo
m = Metro(.5).play()

# contador de pulsos de 3 a 8, 
# dir=2 -> ascendente y descendente
# mul=100 devuelve 300,400,...800
c = Counter(m, min=3, max=8, dir=2, mul=100)

# lo utilizamos como argumento de otra señal
a = Sine(freq=c, mul=.2).mix(2).out()

s.start()
# s.stop()


In [ ]:
# Score(input, fname='event_')
# Llama a las funciones "event_N" incrementando el índice N

from pyo import * ; s = Server(audio="jack",sr=48000).boot(); s.amp=0.2

a = SineLoop(freq=[200,300,400,500], feedback=0.05, mul=.1).out()

def event_0():
    a.freq=[200,300,400,500]
def event_1():
    a.freq=[300,400,450,600]
def event_2():
    a.freq=[150,375,450,525]

m = Metro(1).play()
c = Counter(m, min=0, max=3)

# Score lanza "event_N", con N<-c y c recorre 0,1,2
sc = Score(c)

s.gui(locals())


In [ ]:
# Otro ejemplo

from pyo import * ; s = Server(audio="jack",sr=48000).boot(); s.amp=0.2

adsr=Adsr(attack=.04,decay=.08,sustain=.6,release=.06,dur=.2)
fm = FM(carrier=200, ratio=.248, index=adsr*8, mul=adsr*0.3)
mix = Mix(fm.mix(1), voices=2).out()

def func_0():
    fm.setCarrier(200)
    fm.setIndex(adsr*4)
    adsr.play()

def func_1():
    fm.setCarrier(250)
    fm.setIndex(adsr*6)
    adsr.play()

def func_2():
    fm.setCarrier(300)
    fm.setIndex(adsr*8)
    adsr.play()

# cada 0.2 segundos    
metro = Metro(time=.2).play()
# de 0 a 4, i.e., hay dos pulsos de silencio
counter = Counter(metro, min=0, max=5)
score = Score(counter, fname="func_")

s.gui(locals())


In [ ]:
# Haciendo ritmos con samples
from pyo import * ; s = Server(audio="jack",sr=48000).boot(); s.amp=0.2

sample_paths = ["media/kick.wav","media/snare.wav","media/hihat.wav"]

# create a three-stream SndTable
snd_tabs = SndTable(sample_paths, chnl=0)

# get a list of the durations
durs = snd_tabs.getDur(all=True)

# create triggering lists
kick_list =  [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]
snare_list = [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0]
hat_list   = [0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0]

# put all lists in one list
all_lists = [kick_list, snare_list, hat_list]

# get ms from BPM based on meter and pass it to Metro’s arg
metro = Metro(beatToDur(1/4, 120))

# recorre todas las listas "all_list" de pulsos al tempo marcado por metro
mask = Iter(metro, all_lists)

# create a dummy arithmetic object
beat = metro * mask

# create a three-stream TrigEnv()
player = TrigEnv(beat, snd_tabs, dur=durs, mul=.5).out()
def new_list(l, index=0):
    all_lists[index] = l
    mask.setChoice(all_lists)

metro.play()

s.gui(locals())

In [ ]:
########## Piano ##############

from pyo import *
s = Server(audio="jack",sr=48000, nchnls=2).boot()


####### piano class #########
class Piano:
    '''
    inputs: trig, pitch, dur, mul (pyo objects, polyphonic)
    '''
    def __init__(self, trig, pitch, dur, mul=.3):
        self.trig = trig
        self.pitch = pitch
        self.dur = dur
        self.mul = mul
        self.ta = ExpTable(list=[(0,0),(20,1),(8191,0)], exp=2)
        self.ex = TrigEnv(self.trig, self.ta, dur=self.dur, interp=1)
        self.f = SumOsc(freq=MToF(self.pitch), ratio=1.01, index=.7, mul=self.ex)
        self.ch = Chorus(self.f, depth=.5, feedback=0, bal=.25, mul=self.mul)
        self.rv = Freeverb(self.ch, size=.4, damp=.9, bal=.83, mul=1)
        self.ret = Mix(self.ch+self.rv,2)
    def output(self):
        return self.ret
    def setMul(self, mul):
        self.ch.setMul = mul
    def setDur(self, dur):
        self.ex.setDur = dur
    def setPitch(self, pitch):
        self.f.setFreq = pitch
###### end piano class ######

#### Demo:

tm =.25
mt = Metro([tm,tm*2], poly=10).play()
m = Percent(mt, 50)

pm2 = [33, 36, 38, 40, 43, 45, 48, 50, 52, 55, 57, 60, 62, 64, 67] # A pent min
tc = TrigChoice(m, pm2) # pitch: random notes of a scale

tl = TrigChoice(m, [1,1.5,2.5]) # random choice for note length


pa = Piano(m, tc, tl) # Piano(trig, midi pitch, dur, mul)
    
p = Pan(pa.output(), outs=2, pan=[.3,.7], spread=0)     .out()


#zz= Scope(p)
#################
s.start()
s.gui(locals())

# Música generativa

Brian Eno, Thursday Afternoon

![thursday](thursdayAfternoon.png) 




In [ ]:
# %load ../thursday-afternoon.py
from pyo import *
from random import random

s = Server(audio="jack",midi="jack",sr=48000).boot()

path = "media/thursday/"

# cargamos samples de piano con sus duraciones asociadas
piano = [path + "piano"+str(i)+".wav" for i in range(2,9)]
dursPiano = [36,15,21,15,16,23,60]

# idem con los senos
sine = [path + "sine"+str(i)+".wav" for i in range(1,9)]
dursSine = [36,16,31,21,15,18,19,20]

# los drones no necesitan duraciones
drone = [path + "thursday-drone-"+str(i)+".wav" for i in range(1,4)]

# lanzamos drones en loop
sfDrone = [SfPlayer(drone[i], speed=[1, 1], loop=True, mul=0.1) for i in range(len(drone))]
[d.out() for d in sfDrone]

# creamos players para cada uno de los senos y los pianos
# y arrancamos en silencio (mul=0)
sf = [SfPlayer(p, speed=[1, 1], loop=False, mul=0) for p in piano] +\
     [SfPlayer(s, speed=[1, 1], loop=False, mul=0) for s in sine]
durs = dursPiano + dursSine

# enganchamos players a la salida con un Pattern (mas abajo)
def new_player(i):
    sf[i].out()

# los players entran todos a la vez. Para evitar el "impacto" los 
# silenciamos y hacemos que entren con un fadein de 14 segundos
# variable de amplitud ligada al fader    
amp = Fader(fadein=14).play()

pat = []
for i in range(len(sf)):
    p = Pattern(function=new_player, arg=i, time=durs[i]).play()
    # asociamos la amplitud de los player a la señal del fader amp
    sf[i].mul = amp
    pat.append(p)

s.gui(locals())

